### Aplicações em NLP
## Exercício 1 - CHATBOT

_____

**Grupo**
* Eduardo Bressan Penteado - RA: 101378
* Pedro Ferraresi - RA: 191225

In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

___
DATASET
___

In [ ]:
df = pd.read_csv('../input/questionanswer-dataset/S08_question_answer_pairs.txt', sep='\t')

___
VISUALIZAÇÃO DO DATASET
___

In [ ]:
df.head(10)

In [ ]:
df.columns

In [ ]:
df.info()

___
VERIFICOU-SE QUE AS QUESTÕES ESTÃO DUPLICADAS

- ELIMINAÇÃO DAS QUESTÕES DUPLICADAS

___

In [ ]:
df = df.drop_duplicates( subset='Question' )
df.head(10)

AJUSTE DA COLUNA "ArticleTitle"

- RETIRADA DO UNDERLINE ('_') SUBSTITUINDO-O POR UM ESPAÇO (' ')
___

In [ ]:
df['ArticleTitle'] = df['ArticleTitle'].str.replace('_',' ')
df.head()

___
MESCLA DO DA COLUNA 'ArticleTitle' com a 'Question'
___

In [ ]:
df['Question'] = df['ArticleTitle'] + ' ' + df['Question']
df.head()

___
DATASET APENAS COM AS COLUNAS 'Question' e 'Answer'
___

In [ ]:
df02 = df[['Question', 'Answer']]
df02.head()

___
VERIFICAÇÃO DO DATASET
___

In [ ]:
df02.shape

___
EXCLUSÃO DE INFORMAÇÕES NULAS
___

In [ ]:
df02 = df02.dropna()
df02.shape

Dataset com 2 colunas e 1033 linhas (dados)

___
CÓDIGO PARA AVERIGUAR A SIMULARIDADE DA PERGUNTA SO USUÁRIO COM O BANCO DE PERGUNTAS
___

In [ ]:
print( type( tuple( df['Question'] ) ) )

In [ ]:
stopwords_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    pos_tags = pos_tag(words)
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
#tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(df02['Question']))
print(tfidf_matrix.shape)

___
FUNÇÃO QUE RECEBERÁ A PERGUNTA DO USUÁRIO E SERÁ FEITA A BUSCA PELA PERGUNTA MAIS PRÓXIMA, SUA SIMILARIDADE E EXIBIÇÃO DA RESPOSTA
___

In [ ]:
def pergunta_resposta(pergunta):
    
    query_vect = tfidf_vectorizer.transform([pergunta])
    similaridade = cosine_similarity(query_vect, tfidf_matrix)
    similaridade_max = np.argmax(similaridade)
    
    print('\n> Similarity:\n- {:.2%}\n'.format(similaridade[0, similaridade_max]))
    print('> Nearest Question:\n- {}\n'.format(df02.iloc[similaridade_max]['Question']))
    print('> Answer:\n- {}'.format(df02.iloc[similaridade_max]['Answer']))

___
PERGUNTA DO USUÁRIO E EXIBIÇÃO DA PERGUNTA MAIS PRÓXIMA, SUA SIMILARIDADE E A RESPOSTA PARA ESTA

### **OBS: NECESSÁRIO RODAR A CÉLULA ABAIXO PARA REALIZAÇÃO DA PERGUNTA**
___

In [ ]:
question_user = input('Type your question here: ')
pergunta_resposta(question_user)

___
TESTES COM ALGUMAS PERGUNTAS
___

In [ ]:
pergunta_resposta("Did Lincoln's Wife's Family support slavery?")

In [ ]:
pergunta_resposta("Abraham_Lincoln Did Lincoln's Wife's Family support slavery?")